# Réplication Table 2, 3 et annexes A1 à A6 de Zhang (2021)
## *Pairs trading with general state space models*
### Quantitative Finance, 21(9), 1567-1587

---

**Tables répliquées:**
- Table 2 & 3: Main pairs (PEP-KO, EWT-EWH)
- Table A1: Large Banks + Small Banks
- Table A2: Large × Small Banks
- Table A3: In-Sample / Out-of-Sample Large Banks
- Table A4: In-Sample / Out-of-Sample Small Banks
- Table A5: In-Sample Large × Small Banks
- Table A6: Out-of-Sample Large × Small Banks

## 1. Configuration et Imports

In [ ]:
from __future__ import annotations
import sys
from pathlib import Path
import itertools
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from dataclasses import dataclass
from typing import Tuple, List, Dict
import warnings
import time
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

# Configuration des chemins
PROJECT_ROOT = Path('.').resolve().parent
SRC_DIR = PROJECT_ROOT / "src"
sys.path.insert(0, str(SRC_DIR))
DATA_DIR = PROJECT_ROOT / "data"
DATA_FILE = DATA_DIR / "dataGQ.xlsx"

print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📊 Data file: {DATA_FILE}")

In [3]:
# Numba (optionnel mais recommandé)
try:
    from numba import njit
    NUMBA_AVAILABLE = True
    print("✅ Numba disponible - calculs accélérés")
except ImportError:
    NUMBA_AVAILABLE = False
    print("⚠️  Numba non disponible - calculs plus lents")
    def njit(*args, **kwargs):
        def decorator(func):
            return func
        if len(args) == 1 and callable(args[0]):
            return args[0]
        return decorator

✅ Numba disponible - calculs accélérés


## 2. Univers des Actions et Périodes

In [4]:
# Stock Universes (Zhang 2021 Appendix)
LARGE_BANKS = ['JPM', 'BAC', 'WFC', 'C', 'USB']
SMALL_BANKS = ['CPF', 'BANC', 'CUBI', 'NBHC', 'FCF']
MAIN_PAIRS = [('PEP', 'KO'), ('EWT', 'EWH')]

# Date Ranges (Zhang 2021)
FULL_SAMPLE_START = '2012-01-03'
FULL_SAMPLE_END = '2019-06-28'
EWT_EWH_END = '2019-05-01'

IN_SAMPLE_START = '2012-01-10'
IN_SAMPLE_END = '2018-01-01'
OUT_SAMPLE_START = '2018-01-01'
OUT_SAMPLE_END = '2019-12-01'

print("📈 Large Banks:", LARGE_BANKS)
print("📉 Small Banks:", SMALL_BANKS)
print(f"📅 Full Sample: {FULL_SAMPLE_START} → {FULL_SAMPLE_END}")
print(f"📅 In-Sample: {IN_SAMPLE_START} → {IN_SAMPLE_END}")
print(f"📅 Out-of-Sample: {OUT_SAMPLE_START} → {OUT_SAMPLE_END}")

📈 Large Banks: ['JPM', 'BAC', 'WFC', 'C', 'USB']
📉 Small Banks: ['CPF', 'BANC', 'CUBI', 'NBHC', 'FCF']
📅 Full Sample: 2012-01-03 → 2019-06-28
📅 In-Sample: 2012-01-10 → 2018-01-01
📅 Out-of-Sample: 2018-01-01 → 2019-12-01


## 3. Classes et Fonctions de Base

In [5]:
@dataclass
class PairData:
    """Container for pair price data."""
    PA: pd.Series
    PB: pd.Series
    asset_a: str
    asset_b: str

    @property
    def n_obs(self) -> int:
        return len(self.PA)


@dataclass
class ModelParams:
    """State-space model parameters."""
    theta0: float = 0.0
    theta1: float = 0.95
    theta2: float = 0.0
    q_base: float = 1e-4
    q_het: float = 0.0
    r: float = 1e-4

    @property
    def is_homoscedastic(self) -> bool:
        return self.q_het < 1e-10

In [6]:
def load_pair_data(filepath: str, col_a: str, col_b: str,
                   start_date: str, end_date: str) -> PairData:
    """Load and align pair data from Excel (Yahoo or Bloomberg format)."""
    df = pd.read_excel(filepath)

    if col_a in df.columns:
        if 'Date' in df.columns:
            df = df.set_index('Date')
        elif 'Unnamed: 0' in df.columns:
            df = df.set_index('Unnamed: 0')
        df.index = pd.to_datetime(df.index)
        PA = df[col_a].dropna()
        PB = df[col_b].dropna()
    else:
        # Bloomberg format
        col_a_bb = f'{col_a} US Equity'
        col_b_bb = f'{col_b} US Equity'
        if col_a_bb not in df.columns:
            col_a_bb = f'{col_a} US Equity '
        if col_b_bb not in df.columns:
            col_b_bb = f'{col_b} US Equity '

        def get_series(df, col):
            col_idx = df.columns.get_loc(col)
            date_col = df.columns[col_idx - 1]
            result = pd.DataFrame({
                'date': pd.to_datetime(df[date_col], errors='coerce'),
                'price': pd.to_numeric(df[col], errors='coerce')
            }).dropna().drop_duplicates('date').set_index('date').sort_index()
            return result['price']

        PA = get_series(df, col_a_bb)
        PB = get_series(df, col_b_bb)

    common_idx = PA.index.intersection(PB.index)
    PA, PB = PA.loc[common_idx], PB.loc[common_idx]
    start, end = pd.to_datetime(start_date), pd.to_datetime(end_date)
    mask = (PA.index >= start) & (PA.index <= end)
    return PairData(PA.loc[mask], PB.loc[mask], col_a, col_b)

## 4. Filtres de Kalman (Numba optimisés)

In [7]:
@njit(cache=True)
def halton_sequence_njit(size: int, base: int) -> np.ndarray:
    """Generate Halton sequence."""
    sequence = np.zeros(size)
    for i in range(size):
        n = i + 1
        f, result = 1.0, 0.0
        while n > 0:
            f = f / base
            result = result + f * (n % base)
            n = n // base
        sequence[i] = result
    return sequence


@njit(cache=True)
def kalman_filter_njit(y: np.ndarray, theta0: float, theta1: float,
                       q: float, r: float) -> Tuple[float, np.ndarray]:
    """Kalman Filter for Model I (homoscedastic)."""
    n = len(y)
    if abs(theta1) < 0.999:
        x = theta0 / (1.0 - theta1)
        P = q / (1.0 - theta1 * theta1)
    else:
        x = y[0]
        P = q * 10.0
    
    x_filt = np.zeros(n)
    loglik = 0.0
    log_2pi = np.log(2.0 * np.pi)
    
    for t in range(n):
        if t > 0:
            x = theta0 + theta1 * x
            P = theta1 * theta1 * P + q
        v = y[t] - x
        S = P + r
        if S > 1e-12:
            K = P / S
            x = x + K * v
            P = (1.0 - K) * P
            loglik += -0.5 * (log_2pi + np.log(S) + v * v / S)
        x_filt[t] = x
    return loglik, x_filt


@njit(cache=True)
def qmckf_njit(y: np.ndarray, theta0: float, theta1: float,
               q_base: float, q_het: float, r: float,
               n_particles: int) -> Tuple[float, np.ndarray]:
    """Quasi-Monte Carlo Kalman Filter for Model II (heteroscedastic)."""
    n = len(y)
    x = y[0]
    P = q_base + q_het * x * x
    x_filt = np.zeros(n)
    loglik = 0.0
    log_2pi = np.log(2.0 * np.pi)
    
    h1 = halton_sequence_njit(n_particles, 2)
    h2 = halton_sequence_njit(n_particles, 3)
    for i in range(n_particles):
        h1[i] = max(1e-10, min(1.0 - 1e-10, h1[i]))
        h2[i] = max(1e-10, min(1.0 - 1e-10, h2[i]))
    
    z = np.zeros(n_particles)
    for i in range(n_particles):
        z[i] = np.sqrt(-2.0 * np.log(h1[i])) * np.cos(2.0 * np.pi * h2[i])
    
    samples = np.zeros(n_particles)
    f_samples = np.zeros(n_particles)
    
    for t in range(n):
        if t == 0:
            x_p, P_p = x, P
        else:
            sqrt_P = np.sqrt(max(P, 1e-12))
            sum_f = 0.0
            for i in range(n_particles):
                samples[i] = x + sqrt_P * z[i]
                f_samples[i] = theta0 + theta1 * samples[i]
                sum_f += f_samples[i]
            x_p = sum_f / n_particles
            sum_var, sum_g = 0.0, 0.0
            for i in range(n_particles):
                diff = f_samples[i] - x_p
                sum_var += diff * diff
                sum_g += q_base + q_het * samples[i] * samples[i]
            P_p = sum_var / n_particles + sum_g / n_particles
        
        v = y[t] - x_p
        S = P_p + r
        if S > 1e-12:
            K = P_p / S
            x = x_p + K * v
            P = (1.0 - K) * P_p
            loglik += -0.5 * (log_2pi + np.log(S) + v * v / S)
        else:
            x, P = x_p, P_p
        x_filt[t] = x
    return loglik, x_filt

## 5. Stratégies de Trading

In [8]:
@njit(cache=True)
def strategy_A_njit(x: np.ndarray, U: np.ndarray, L: np.ndarray, C: float) -> np.ndarray:
    """Strategy A: Enter at boundaries, exit at mean."""
    n = len(x)
    sig = np.zeros(n)
    pos = 0
    for t in range(n):
        if pos == 0:
            if x[t] >= U[t]:
                pos = -1
            elif x[t] <= L[t]:
                pos = 1
        elif pos == 1 and x[t] >= C:
            pos = 0
        elif pos == -1 and x[t] <= C:
            pos = 0
        sig[t] = pos
    return sig


@njit(cache=True)
def strategy_C_njit(x: np.ndarray, U: np.ndarray, L: np.ndarray, C: float) -> np.ndarray:
    """Strategy C: Re-entry with stop-loss (Zhang's main strategy)."""
    n = len(x)
    sig = np.zeros(n)
    pos = 0
    for t in range(1, n):
        prev, curr = x[t - 1], x[t]
        U_prev, U_curr = U[t - 1], U[t]
        L_prev, L_curr = L[t - 1], L[t]
        
        entry_short = (prev > U_prev) and (curr <= U_curr)
        entry_long = (prev < L_prev) and (curr >= L_curr)
        exit_long = (prev < C) and (curr >= C)
        exit_short = (prev > C) and (curr <= C)
        stop_short = (prev < U_prev) and (curr >= U_curr)
        stop_long = (prev > L_prev) and (curr <= L_curr)
        
        if pos == 0:
            if entry_short:
                pos = -1
            elif entry_long:
                pos = 1
        elif pos == 1 and (exit_long or stop_long):
            pos = 0
        elif pos == -1 and (exit_short or stop_short):
            pos = 0
        sig[t] = pos
    return sig


@njit(cache=True)
def compute_thresholds_njit(x_filt: np.ndarray, q_base: float, q_het: float,
                            n_std: float, is_hetero: bool) -> Tuple[np.ndarray, np.ndarray, float]:
    """Compute trading thresholds."""
    n = len(x_filt)
    C = np.mean(x_filt)
    sigma_emp = np.std(x_filt)
    U = np.zeros(n)
    L = np.zeros(n)
    
    if is_hetero and q_het > 1e-10:
        g_x = np.sqrt(q_base + q_het * x_filt * x_filt)
        mean_g = np.mean(g_x)
        for t in range(n):
            sigma_t = g_x[t] / mean_g * sigma_emp
            U[t] = C + n_std * sigma_t
            L[t] = C - n_std * sigma_t
    else:
        threshold = n_std * sigma_emp
        for t in range(n):
            U[t] = C + threshold
            L[t] = C - threshold
    return U, L, C


@njit(cache=True)
def backtest_njit(signals: np.ndarray, x_filt: np.ndarray, cost_bp: float) -> Tuple[float, float, int]:
    """Backtest. Returns (annualized_return, sharpe, n_trades)."""
    n = len(signals)
    pnl = np.zeros(n)
    n_trades = 0
    cost_factor = 2.0 * cost_bp / 10000.0
    
    for t in range(1, n):
        dx = x_filt[t] - x_filt[t - 1]
        pos_change = abs(signals[t] - signals[t - 1])
        if pos_change > 0:
            n_trades += 1
        pnl[t] = signals[t] * dx - pos_change * cost_factor
    
    cum_pnl = np.sum(pnl)
    ann_ret = cum_pnl / (n / 252.0)
    mean_pnl = np.mean(pnl)
    std_pnl = np.std(pnl)
    ann_std = std_pnl * np.sqrt(252.0)
    
    if ann_std > 1e-10:
        sharpe = (ann_ret - 0.02) / ann_std
    else:
        sharpe = 0.0
    return ann_ret, sharpe, n_trades


@njit(cache=True)
def grid_search_njit(x_filt: np.ndarray, q_base: float, q_het: float,
                     is_hetero: bool, use_strategy_C: bool, cost_bp: float) -> Tuple[float, float, float, int]:
    """Grid search for optimal threshold."""
    best_n = 1.0
    best_ret = -1e10
    best_sr = -1e10
    best_trades = 0
    
    for i in range(25):
        n_std = 0.1 + i * 0.1
        U, L, C = compute_thresholds_njit(x_filt, q_base, q_het, n_std, is_hetero)
        
        if use_strategy_C:
            sig = strategy_C_njit(x_filt, U, L, C)
        else:
            sig = strategy_A_njit(x_filt, U, L, C)
        
        ann_ret, sharpe, n_trades = backtest_njit(sig, x_filt, cost_bp)
        
        if n_trades > 0 and sharpe > best_sr:
            best_sr = sharpe
            best_ret = ann_ret
            best_n = n_std
            best_trades = n_trades
    
    return best_n, best_ret, best_sr, best_trades

## 6. Estimation MLE

In [9]:
def estimate_gamma_ols(log_PA: np.ndarray, log_PB: np.ndarray) -> float:
    """Estimate hedge ratio γ via OLS."""
    X = np.column_stack([np.ones(len(log_PB)), log_PB])
    return float(np.linalg.lstsq(X, log_PA, rcond=None)[0][1])


def estimate_model_I(y: np.ndarray) -> Tuple[ModelParams, np.ndarray, float]:
    """Estimate Model I parameters by MLE."""
    y_mean, y_var = np.mean(y), np.var(y)
    rho = np.corrcoef(y[:-1] - y_mean, y[1:] - y_mean)[0, 1]
    theta1_init = float(np.clip(rho, 0.8, 0.99))

    z0 = np.array([y_mean * (1 - theta1_init), np.arctanh(theta1_init),
                   np.log(y_var * (1 - theta1_init ** 2) * 0.7 + 1e-10),
                   np.log(y_var * 0.3 + 1e-10)])

    def neg_ll(z):
        try:
            ll, _ = kalman_filter_njit(y, z[0], np.tanh(z[1]), np.exp(z[2]), np.exp(z[3]))
            return -ll if np.isfinite(ll) else 1e10
        except:
            return 1e10

    bounds = [(-0.5, 0.5), (np.arctanh(0.5), np.arctanh(0.999)),
              (np.log(1e-8), np.log(1.0)), (np.log(1e-8), np.log(1.0))]
    res = minimize(neg_ll, z0, method='L-BFGS-B', bounds=bounds)
    params = ModelParams(theta0=res.x[0], theta1=np.tanh(res.x[1]),
                         q_base=np.exp(res.x[2]), r=np.exp(res.x[3]))
    ll, x_filt = kalman_filter_njit(y, params.theta0, params.theta1, params.q_base, params.r)
    return params, x_filt, ll


def estimate_model_II(y: np.ndarray) -> Tuple[ModelParams, np.ndarray, float]:
    """Estimate Model II parameters by MLE with QMCKF."""
    y_mean = np.mean(y)
    best_ll, best_params, best_filt = -np.inf, None, None

    for t0, t1, q_b, q_h, r in [(y_mean * 0.01, 0.95, 0.0005, 0.10, 0.010),
                                (y_mean * 0.01, 0.93, 0.0003, 0.13, 0.011),
                                (y_mean * 0.01, 0.96, 0.0010, 0.08, 0.008)]:
        z0 = np.array([t0, np.arctanh(t1), np.log(q_b), np.log(q_h), np.log(r)])

        def neg_ll(z):
            try:
                ll, _ = qmckf_njit(y, z[0], np.tanh(z[1]), np.exp(z[2]), np.exp(z[3]), np.exp(z[4]), 50)
                return -ll if np.isfinite(ll) else 1e10
            except:
                return 1e10

        bounds = [(-0.1, 0.1), (np.arctanh(0.85), np.arctanh(0.99)),
                  (np.log(1e-6), np.log(0.005)), (np.log(0.05), np.log(0.3)),
                  (np.log(0.005), np.log(0.05))]
        try:
            res = minimize(neg_ll, z0, method='L-BFGS-B', bounds=bounds, options={'maxiter': 500})
            params = ModelParams(theta0=res.x[0], theta1=np.tanh(res.x[1]),
                                 q_base=np.exp(res.x[2]), q_het=np.exp(res.x[3]), r=np.exp(res.x[4]))
            ll, x_filt = qmckf_njit(y, params.theta0, params.theta1,
                                    params.q_base, params.q_het, params.r, 100)
            if ll > best_ll:
                best_ll, best_params, best_filt = ll, params, x_filt
        except:
            continue

    if best_params is None:
        best_params = ModelParams(theta0=0.0, theta1=0.95, q_base=0.0003, q_het=0.1, r=0.01)
        best_ll, best_filt = qmckf_njit(y, 0.0, 0.95, 0.0003, 0.1, 0.01, 100)
    return best_params, best_filt, best_ll

## 7. Analyse des Paires

In [10]:
def analyze_pair(pair: PairData, cost_bp: float = 20.0) -> Dict:
    """Analyze a pair: Model I + Strategy A, Model II + Strategy C."""
    log_PA, log_PB = np.log(pair.PA.values), np.log(pair.PB.values)
    gamma = estimate_gamma_ols(log_PA, log_PB)
    y = log_PA - gamma * log_PB

    # Model I + Strategy A
    p1, f1, _ = estimate_model_I(y)
    _, ret_m1, sr_m1, _ = grid_search_njit(f1, p1.q_base, 0.0, False, False, cost_bp)

    # Model II + Strategy C
    p2, f2, _ = estimate_model_II(y)
    _, ret_m2, sr_m2, _ = grid_search_njit(f2, p2.q_base, p2.q_het, True, True, cost_bp)

    return {
        'Stock #1': pair.asset_a,
        'Stock #2': pair.asset_b,
        'M1_Return': ret_m1,
        'M1_Sharpe': sr_m1,
        'M2_Return': ret_m2,
        'M2_Sharpe': sr_m2,
        'Imp_Return': (ret_m2 / ret_m1 - 1) * 100 if abs(ret_m1) > 1e-6 else 0,
        'Imp_Sharpe': (sr_m2 / sr_m1 - 1) * 100 if abs(sr_m1) > 1e-6 else 0,
    }


def analyze_pair_is_oos(filepath: str, col_a: str, col_b: str,
                        is_start: str, is_end: str,
                        oos_start: str, oos_end: str,
                        cost_bp: float = 20.0) -> Tuple[Dict, Dict]:
    """Analyze pair with in-sample and out-of-sample periods."""
    # In-Sample
    pair_is = load_pair_data(filepath, col_a, col_b, is_start, is_end)
    log_PA_is, log_PB_is = np.log(pair_is.PA.values), np.log(pair_is.PB.values)
    gamma = estimate_gamma_ols(log_PA_is, log_PB_is)
    y_is = log_PA_is - gamma * log_PB_is
    
    p1, f1_is, _ = estimate_model_I(y_is)
    _, ret_m1_is, sr_m1_is, _ = grid_search_njit(f1_is, p1.q_base, 0.0, False, False, cost_bp)
    
    p2, f2_is, _ = estimate_model_II(y_is)
    _, ret_m2_is, sr_m2_is, _ = grid_search_njit(f2_is, p2.q_base, p2.q_het, True, True, cost_bp)
    
    result_is = {
        'Stock #1': col_a, 'Stock #2': col_b,
        'M1_Return': ret_m1_is, 'M1_Sharpe': sr_m1_is,
        'M2_Return': ret_m2_is, 'M2_Sharpe': sr_m2_is,
        'Imp_Return': (ret_m2_is / ret_m1_is - 1) * 100 if abs(ret_m1_is) > 1e-6 else 0,
        'Imp_Sharpe': (sr_m2_is / sr_m1_is - 1) * 100 if abs(sr_m1_is) > 1e-6 else 0,
    }
    
    # Out-of-Sample (use IS parameters)
    pair_oos = load_pair_data(filepath, col_a, col_b, oos_start, oos_end)
    log_PA_oos, log_PB_oos = np.log(pair_oos.PA.values), np.log(pair_oos.PB.values)
    y_oos = log_PA_oos - gamma * log_PB_oos
    
    _, f1_oos = kalman_filter_njit(y_oos, p1.theta0, p1.theta1, p1.q_base, p1.r)
    _, ret_m1_oos, sr_m1_oos, _ = grid_search_njit(f1_oos, p1.q_base, 0.0, False, False, cost_bp)
    
    _, f2_oos = qmckf_njit(y_oos, p2.theta0, p2.theta1, p2.q_base, p2.q_het, p2.r, 100)
    _, ret_m2_oos, sr_m2_oos, _ = grid_search_njit(f2_oos, p2.q_base, p2.q_het, True, True, cost_bp)
    
    result_oos = {
        'Stock #1': col_a, 'Stock #2': col_b,
        'M1_Return': ret_m1_oos, 'M1_Sharpe': sr_m1_oos,
        'M2_Return': ret_m2_oos, 'M2_Sharpe': sr_m2_oos,
        'Imp_Return': (ret_m2_oos / ret_m1_oos - 1) * 100 if abs(ret_m1_oos) > 1e-6 else 0,
        'Imp_Sharpe': (sr_m2_oos / sr_m1_oos - 1) * 100 if abs(sr_m1_oos) > 1e-6 else 0,
    }
    
    return result_is, result_oos

## 8. Fonctions d'affichage (format Zhang)

In [11]:
def format_zhang_table(df: pd.DataFrame, title: str, note: str = "") -> pd.DataFrame:
    """Format DataFrame in Zhang (2021) style."""
    # Rename columns
    df_display = df.copy()
    df_display = df_display.rename(columns={
        'M1_Return': 'Return',
        'M1_Sharpe': 'Sharpe',
        'M2_Return': 'Return ',
        'M2_Sharpe': 'Sharpe ',
        'Imp_Return': 'Return  ',
        'Imp_Sharpe': 'Sharpe  '
    })
    
    # Add Pair column
    df_display.insert(0, 'Pair', range(1, len(df_display) + 1))
    
    # Add summary rows
    numeric_cols = ['Return', 'Sharpe', 'Return ', 'Sharpe ', 'Return  ', 'Sharpe  ']
    summary = pd.DataFrame({
        'Pair': ['Mean', 'Min', 'Max', 'Median'],
        'Stock #1': ['', '', '', ''],
        'Stock #2': ['', '', '', ''],
    })
    
    for col in numeric_cols:
        if col in df_display.columns:
            summary[col] = [
                df_display[col].mean(),
                df_display[col].min(),
                df_display[col].max(),
                df_display[col].median()
            ]
    
    df_final = pd.concat([df_display, summary], ignore_index=True)
    
    return df_final


def display_zhang_table(df: pd.DataFrame, title: str, page: int = 0, note: str = ""):
    """Display table in Zhang (2021) style."""
    header = f"<h3>{title}</h3>"
    if page > 0:
        header = f"<div style='text-align:right'>G. Zhang &nbsp;&nbsp; {page}</div>" + header
    
    # Create multi-level header
    html = f"""
    {header}
    <table style='border-collapse: collapse; width: 100%; font-size: 12px;'>
        <tr style='border-bottom: 2px solid black;'>
            <th colspan='3'></th>
            <th colspan='2' style='text-align:center;'>Model I + Strategy A</th>
            <th colspan='2' style='text-align:center;'>Model II + Strategy C</th>
            <th colspan='2' style='text-align:center;'>Improvement (in %)</th>
        </tr>
        <tr style='border-bottom: 1px solid black;'>
            <th>Pair</th>
            <th>Stock #1</th>
            <th>Stock #2</th>
            <th>Return</th>
            <th>Sharpe</th>
            <th>Return</th>
            <th>Sharpe</th>
            <th>Return</th>
            <th>Sharpe</th>
        </tr>
    """
    
    for _, row in df.iterrows():
        pair = row.get('Pair', '')
        s1 = row.get('Stock #1', '')
        s2 = row.get('Stock #2', '')
        m1_ret = row.get('M1_Return', row.get('Return', 0))
        m1_sr = row.get('M1_Sharpe', row.get('Sharpe', 0))
        m2_ret = row.get('M2_Return', row.get('Return ', 0))
        m2_sr = row.get('M2_Sharpe', row.get('Sharpe ', 0))
        imp_ret = row.get('Imp_Return', row.get('Return  ', 0))
        imp_sr = row.get('Imp_Sharpe', row.get('Sharpe  ', 0))
        
        style = "border-top: 1px solid black;" if pair in ['Mean', 'Min', 'Max', 'Median'] else ""
        
        html += f"""
        <tr style='{style}'>
            <td>{pair}</td>
            <td>{s1}</td>
            <td>{s2}</td>
            <td style='text-align:right;'>{m1_ret:.4f}</td>
            <td style='text-align:right;'>{m1_sr:.4f}</td>
            <td style='text-align:right;'>{m2_ret:.4f}</td>
            <td style='text-align:right;'>{m2_sr:.4f}</td>
            <td style='text-align:right;'>{imp_ret:.2f}</td>
            <td style='text-align:right;'>{imp_sr:.2f}</td>
        </tr>
        """
    
    html += "</table>"
    
    if note:
        html += f"<p style='font-size:10px;'><i>Note: {note}</i></p>"
    
    display(HTML(html))

## 9. Warm-up JIT Compilation

In [12]:
# Warm-up JIT compilation
if NUMBA_AVAILABLE:
    print("⏳ JIT Compilation...")
    dummy = np.random.randn(100)
    _ = kalman_filter_njit(dummy, 0.0, 0.95, 0.001, 0.001)
    _ = qmckf_njit(dummy, 0.0, 0.95, 0.001, 0.1, 0.01, 50)
    print("✅ JIT Compilation done!")

⏳ JIT Compilation...
✅ JIT Compilation done!


---
# Tables 2 & 3: Main Pairs (PEP-KO, EWT-EWH)

In [13]:
print("📊 Analyzing Main Pairs...")
main_results = []

for col_a, col_b in MAIN_PAIRS:
    end_date = EWT_EWH_END if col_a == 'EWT' else FULL_SAMPLE_END
    try:
        pair = load_pair_data(DATA_FILE, col_a, col_b, FULL_SAMPLE_START, end_date)
        print(f"  {col_a}-{col_b}: {pair.n_obs} observations")
        result = analyze_pair(pair)
        main_results.append(result)
    except Exception as e:
        print(f"  ❌ {col_a}-{col_b}: Error - {e}")

df_main = pd.DataFrame(main_results)
display_zhang_table(format_zhang_table(df_main, ""), 
                    "Table 2 & 3. Performance of Pairs Trading on Main Pairs",
                    note="Return is the annualized return, displayed in decimal. Sharpe is the annualized Sharpe ratio.")

📊 Analyzing Main Pairs...
  PEP-KO: 1884 observations
  EWT-EWH: 1843 observations


---
# Table A1: Pairs of Large Banks and Small Banks

In [14]:
print("📊 Analyzing Large Banks...")
large_pairs = list(itertools.combinations(LARGE_BANKS, 2))
results_large = []

for s1, s2 in large_pairs:
    try:
        pair = load_pair_data(DATA_FILE, s1, s2, FULL_SAMPLE_START, FULL_SAMPLE_END)
        result = analyze_pair(pair)
        results_large.append(result)
        print(f"  {s1}-{s2}: SR(M1)={result['M1_Sharpe']:.4f}, SR(M2)={result['M2_Sharpe']:.4f}")
    except Exception as e:
        print(f"  ❌ {s1}-{s2}: Error - {e}")

df_large = pd.DataFrame(results_large)
display_zhang_table(format_zhang_table(df_large, ""),
                    "Table A1 - Panel A: Pairs of Large Banks", page=1582,
                    note="Return is the annualized return, displayed in decimal. Sharpe is the annualized Sharpe ratio.")

📊 Analyzing Large Banks...
  JPM-BAC: SR(M1)=0.6002, SR(M2)=1.0097
  JPM-WFC: SR(M1)=-0.0829, SR(M2)=0.7803
  JPM-C: SR(M1)=0.7500, SR(M2)=0.8270
  JPM-USB: SR(M1)=0.6991, SR(M2)=1.1117
  BAC-WFC: SR(M1)=0.0283, SR(M2)=1.1061
  BAC-C: SR(M1)=0.6713, SR(M2)=1.5029
  BAC-USB: SR(M1)=0.4735, SR(M2)=1.8097
  WFC-C: SR(M1)=1.0345, SR(M2)=0.9297
  WFC-USB: SR(M1)=0.1099, SR(M2)=0.5579
  C-USB: SR(M1)=0.5278, SR(M2)=1.5589


In [15]:
print("📊 Analyzing Small Banks...")
small_pairs = list(itertools.combinations(SMALL_BANKS, 2))
results_small = []

for s1, s2 in small_pairs:
    try:
        pair = load_pair_data(DATA_FILE, s1, s2, FULL_SAMPLE_START, FULL_SAMPLE_END)
        result = analyze_pair(pair)
        results_small.append(result)
        print(f"  {s1}-{s2}: SR(M1)={result['M1_Sharpe']:.4f}, SR(M2)={result['M2_Sharpe']:.4f}")
    except Exception as e:
        print(f"  ❌ {s1}-{s2}: Error - {e}")

df_small = pd.DataFrame(results_small)
if not df_small.empty:
    display_zhang_table(format_zhang_table(df_small, ""),
                        "Table A1 - Panel B: Pairs of Small Banks", page=1582,
                        note="Return is the annualized return, displayed in decimal. Sharpe is the annualized Sharpe ratio.")

📊 Analyzing Small Banks...
  CPF-BANC: SR(M1)=0.4346, SR(M2)=1.4414
  CPF-CUBI: SR(M1)=0.2455, SR(M2)=1.5663
  CPF-NBHC: SR(M1)=0.7210, SR(M2)=1.0260
  CPF-FCF: SR(M1)=0.4552, SR(M2)=1.2666
  BANC-CUBI: SR(M1)=0.6502, SR(M2)=1.5147
  BANC-NBHC: SR(M1)=0.3448, SR(M2)=1.2835
  BANC-FCF: SR(M1)=0.4070, SR(M2)=1.3768
  CUBI-NBHC: SR(M1)=0.4980, SR(M2)=1.1451
  CUBI-FCF: SR(M1)=0.3313, SR(M2)=0.9409
  NBHC-FCF: SR(M1)=0.5937, SR(M2)=1.4648


---
# Table A2: Pairs Between Large and Small Banks

In [16]:
print("📊 Analyzing Large × Small Banks...")
cross_pairs = list(itertools.product(LARGE_BANKS, SMALL_BANKS))
results_cross = []

for s1, s2 in cross_pairs:
    try:
        pair = load_pair_data(DATA_FILE, s1, s2, FULL_SAMPLE_START, FULL_SAMPLE_END)
        result = analyze_pair(pair)
        results_cross.append(result)
        print(f"  {s1}-{s2}: SR(M1)={result['M1_Sharpe']:.4f}, SR(M2)={result['M2_Sharpe']:.4f}")
    except Exception as e:
        print(f"  ❌ {s1}-{s2}: Error - {e}")

df_cross = pd.DataFrame(results_cross)
if not df_cross.empty:
    display_zhang_table(format_zhang_table(df_cross, ""),
                        "Table A2. Performance of Pairs Trading on Pairs Between Large Banks and Small Banks", 
                        page=1583,
                        note="Return is the annualized return, displayed in decimal. Sharpe is the annualized Sharpe ratio.")

📊 Analyzing Large × Small Banks...
  JPM-CPF: SR(M1)=0.4254, SR(M2)=1.2301
  JPM-BANC: SR(M1)=0.6012, SR(M2)=1.0845
  JPM-CUBI: SR(M1)=0.0060, SR(M2)=0.9446
  JPM-NBHC: SR(M1)=1.1577, SR(M2)=1.4952
  JPM-FCF: SR(M1)=0.3168, SR(M2)=1.5338
  BAC-CPF: SR(M1)=0.6365, SR(M2)=1.4427
  BAC-BANC: SR(M1)=0.9263, SR(M2)=0.9850
  BAC-CUBI: SR(M1)=0.4475, SR(M2)=0.9608
  BAC-NBHC: SR(M1)=0.8576, SR(M2)=1.5523
  BAC-FCF: SR(M1)=0.8640, SR(M2)=1.4668
  WFC-CPF: SR(M1)=0.4333, SR(M2)=0.9462
  WFC-BANC: SR(M1)=1.0897, SR(M2)=1.0740
  WFC-CUBI: SR(M1)=0.5341, SR(M2)=1.3322
  WFC-NBHC: SR(M1)=1.1738, SR(M2)=1.1533
  WFC-FCF: SR(M1)=0.3850, SR(M2)=0.7792
  C-CPF: SR(M1)=0.8814, SR(M2)=1.5036
  C-BANC: SR(M1)=0.6291, SR(M2)=0.9894
  C-CUBI: SR(M1)=0.5827, SR(M2)=1.2423
  C-NBHC: SR(M1)=1.0026, SR(M2)=1.4609
  C-FCF: SR(M1)=0.5826, SR(M2)=1.1415
  USB-CPF: SR(M1)=0.6241, SR(M2)=1.1358
  USB-BANC: SR(M1)=0.7781, SR(M2)=0.8892
  USB-CUBI: SR(M1)=0.1251, SR(M2)=1.0571
  USB-NBHC: SR(M1)=0.4597, SR(M2)=0.8442


---
# Table A3: In-Sample and Out-of-Sample Performance on Large Banks

In [17]:
print("📊 Analyzing Large Banks (IS/OOS)...")
results_a3_is, results_a3_oos = [], []

for s1, s2 in large_pairs:
    try:
        r_is, r_oos = analyze_pair_is_oos(DATA_FILE, s1, s2,
                                          IN_SAMPLE_START, IN_SAMPLE_END,
                                          OUT_SAMPLE_START, OUT_SAMPLE_END)
        results_a3_is.append(r_is)
        results_a3_oos.append(r_oos)
        print(f"  {s1}-{s2}: IS SR={r_is['M2_Sharpe']:.4f}, OOS SR={r_oos['M2_Sharpe']:.4f}")
    except Exception as e:
        print(f"  ❌ {s1}-{s2}: Error - {e}")

df_a3_is = pd.DataFrame(results_a3_is)
df_a3_oos = pd.DataFrame(results_a3_oos)

if not df_a3_is.empty:
    display_zhang_table(format_zhang_table(df_a3_is, ""),
                        "Table A3 - Panel A: In Sample Performance on Pairs of Large Banks", 
                        page=1584,
                        note=f"The data is from {IN_SAMPLE_START} to {IN_SAMPLE_END}.")

if not df_a3_oos.empty:
    display_zhang_table(format_zhang_table(df_a3_oos, ""),
                        "Table A3 - Panel B: Out of Sample Performance on Pairs of Large Banks", 
                        page=1584,
                        note=f"The data is from {OUT_SAMPLE_START} to {OUT_SAMPLE_END}.")

📊 Analyzing Large Banks (IS/OOS)...
  JPM-BAC: IS SR=0.7943, OOS SR=0.6630
  JPM-WFC: IS SR=0.9286, OOS SR=1.5864
  JPM-C: IS SR=0.4345, OOS SR=2.0927
  JPM-USB: IS SR=1.2805, OOS SR=2.1821
  BAC-WFC: IS SR=1.3250, OOS SR=1.7203
  BAC-C: IS SR=1.3175, OOS SR=2.1218
  BAC-USB: IS SR=1.7244, OOS SR=2.0028
  WFC-C: IS SR=1.2573, OOS SR=1.8825
  WFC-USB: IS SR=-0.0425, OOS SR=0.9730
  C-USB: IS SR=1.2575, OOS SR=1.7026


---
# Table A4: In-Sample and Out-of-Sample Performance on Small Banks

In [18]:
print("📊 Analyzing Small Banks (IS/OOS)...")
results_a4_is, results_a4_oos = [], []

for s1, s2 in small_pairs:
    try:
        r_is, r_oos = analyze_pair_is_oos(DATA_FILE, s1, s2,
                                          IN_SAMPLE_START, IN_SAMPLE_END,
                                          OUT_SAMPLE_START, OUT_SAMPLE_END)
        results_a4_is.append(r_is)
        results_a4_oos.append(r_oos)
        print(f"  {s1}-{s2}: IS SR={r_is['M2_Sharpe']:.4f}, OOS SR={r_oos['M2_Sharpe']:.4f}")
    except Exception as e:
        print(f"  ❌ {s1}-{s2}: Error - {e}")

df_a4_is = pd.DataFrame(results_a4_is)
df_a4_oos = pd.DataFrame(results_a4_oos)

if not df_a4_is.empty:
    display_zhang_table(format_zhang_table(df_a4_is, ""),
                        "Table A4 - Panel A: In Sample Performance on Pairs of Small Banks", 
                        page=1585,
                        note=f"The data is from {IN_SAMPLE_START} to {IN_SAMPLE_END}.")

if not df_a4_oos.empty:
    display_zhang_table(format_zhang_table(df_a4_oos, ""),
                        "Table A4 - Panel B: Out of Sample Performance on Pairs of Small Banks", 
                        page=1585,
                        note=f"The data is from {OUT_SAMPLE_START} to {OUT_SAMPLE_END}.")

📊 Analyzing Small Banks (IS/OOS)...
  CPF-BANC: IS SR=1.2735, OOS SR=1.1844
  CPF-CUBI: IS SR=1.7639, OOS SR=1.3682
  CPF-NBHC: IS SR=1.3074, OOS SR=1.2380
  CPF-FCF: IS SR=1.9088, OOS SR=1.2142
  BANC-CUBI: IS SR=1.3778, OOS SR=2.4242
  BANC-NBHC: IS SR=0.5201, OOS SR=1.2105
  BANC-FCF: IS SR=1.6356, OOS SR=1.4546
  CUBI-NBHC: IS SR=0.7800, OOS SR=1.4032
  CUBI-FCF: IS SR=1.2452, OOS SR=1.3488
  NBHC-FCF: IS SR=1.6888, OOS SR=1.3871


---
# Table A5: In-Sample Performance on Large × Small Banks

In [19]:
print("📊 Analyzing Large × Small Banks (IS)...")
results_a5 = []

for s1, s2 in cross_pairs:
    try:
        r_is, _ = analyze_pair_is_oos(DATA_FILE, s1, s2,
                                      IN_SAMPLE_START, IN_SAMPLE_END,
                                      OUT_SAMPLE_START, OUT_SAMPLE_END)
        results_a5.append(r_is)
        print(f"  {s1}-{s2}: SR(M2)={r_is['M2_Sharpe']:.4f}")
    except Exception as e:
        print(f"  ❌ {s1}-{s2}: Error - {e}")

df_a5 = pd.DataFrame(results_a5)
if not df_a5.empty:
    display_zhang_table(format_zhang_table(df_a5, ""),
                        "Table A5. In Sample Performance of Pairs Trading on Pairs Between Large Banks and Small Banks", 
                        page=1586,
                        note=f"The data is from {IN_SAMPLE_START} to {IN_SAMPLE_END}.")

📊 Analyzing Large × Small Banks (IS)...
  JPM-CPF: SR(M2)=1.7906
  JPM-BANC: SR(M2)=1.2798
  JPM-CUBI: SR(M2)=1.1355
  JPM-NBHC: SR(M2)=1.3657
  JPM-FCF: SR(M2)=1.9394
  BAC-CPF: SR(M2)=1.4710
  BAC-BANC: SR(M2)=1.0377
  BAC-CUBI: SR(M2)=1.4439
  BAC-NBHC: SR(M2)=1.7457
  BAC-FCF: SR(M2)=1.4311
  WFC-CPF: SR(M2)=0.8442
  WFC-BANC: SR(M2)=1.0421
  WFC-CUBI: SR(M2)=1.2455
  WFC-NBHC: SR(M2)=0.9480
  WFC-FCF: SR(M2)=0.8021
  C-CPF: SR(M2)=1.2969
  C-BANC: SR(M2)=1.0566
  C-CUBI: SR(M2)=1.6023
  C-NBHC: SR(M2)=1.3456
  C-FCF: SR(M2)=1.1307
  USB-CPF: SR(M2)=1.1586
  USB-BANC: SR(M2)=0.9508
  USB-CUBI: SR(M2)=1.2656
  USB-NBHC: SR(M2)=1.0023
  USB-FCF: SR(M2)=1.2090


---
# Table A6: Out-of-Sample Performance on Large × Small Banks

In [20]:
print("📊 Analyzing Large × Small Banks (OOS)...")
results_a6 = []

for s1, s2 in cross_pairs:
    try:
        _, r_oos = analyze_pair_is_oos(DATA_FILE, s1, s2,
                                       IN_SAMPLE_START, IN_SAMPLE_END,
                                       OUT_SAMPLE_START, OUT_SAMPLE_END)
        results_a6.append(r_oos)
        print(f"  {s1}-{s2}: SR(M2)={r_oos['M2_Sharpe']:.4f}")
    except Exception as e:
        print(f"  ❌ {s1}-{s2}: Error - {e}")

df_a6 = pd.DataFrame(results_a6)
if not df_a6.empty:
    display_zhang_table(format_zhang_table(df_a6, ""),
                        "Table A6. Out of Sample Performance of Pairs Trading on Pairs Between Large Banks and Small Banks", 
                        page=1587,
                        note=f"The data is from {OUT_SAMPLE_START} to {OUT_SAMPLE_END}.")

📊 Analyzing Large × Small Banks (OOS)...
  JPM-CPF: SR(M2)=1.8992
  JPM-BANC: SR(M2)=1.4055
  JPM-CUBI: SR(M2)=1.3906
  JPM-NBHC: SR(M2)=1.6330
  JPM-FCF: SR(M2)=1.0034
  BAC-CPF: SR(M2)=2.0997
  BAC-BANC: SR(M2)=2.0479
  BAC-CUBI: SR(M2)=1.4217
  BAC-NBHC: SR(M2)=2.3696
  BAC-FCF: SR(M2)=0.8585
  WFC-CPF: SR(M2)=1.8138
  WFC-BANC: SR(M2)=2.4310
  WFC-CUBI: SR(M2)=2.0321
  WFC-NBHC: SR(M2)=1.8905
  WFC-FCF: SR(M2)=1.8322
  C-CPF: SR(M2)=2.1518
  C-BANC: SR(M2)=2.0597
  C-CUBI: SR(M2)=1.8881
  C-NBHC: SR(M2)=2.1989
  C-FCF: SR(M2)=1.4219
  USB-CPF: SR(M2)=1.9530
  USB-BANC: SR(M2)=1.6552
  USB-CUBI: SR(M2)=1.8249
  USB-NBHC: SR(M2)=1.9446
  USB-FCF: SR(M2)=1.0290


---
# Résumé Final

In [21]:
print("\n" + "="*80)
print("RÉSUMÉ FINAL")
print("="*80)

all_tables = {
    'Table 2&3 (Main)': df_main,
    'Table A1 (Large Banks)': df_large,
    'Table A1 (Small Banks)': df_small if 'df_small' in dir() and not df_small.empty else pd.DataFrame(),
    'Table A2 (Large×Small)': df_cross if 'df_cross' in dir() and not df_cross.empty else pd.DataFrame(),
    'Table A3 IS': df_a3_is if 'df_a3_is' in dir() and not df_a3_is.empty else pd.DataFrame(),
    'Table A3 OOS': df_a3_oos if 'df_a3_oos' in dir() and not df_a3_oos.empty else pd.DataFrame(),
    'Table A4 IS': df_a4_is if 'df_a4_is' in dir() and not df_a4_is.empty else pd.DataFrame(),
    'Table A4 OOS': df_a4_oos if 'df_a4_oos' in dir() and not df_a4_oos.empty else pd.DataFrame(),
    'Table A5 (IS)': df_a5 if 'df_a5' in dir() and not df_a5.empty else pd.DataFrame(),
    'Table A6 (OOS)': df_a6 if 'df_a6' in dir() and not df_a6.empty else pd.DataFrame(),
}

for name, df in all_tables.items():
    if not df.empty:
        print(f"\n{name}:")
        print(f"  Pairs: {len(df)}")
        print(f"  Mean M1 Sharpe: {df['M1_Sharpe'].mean():.4f}")
        print(f"  Mean M2 Sharpe: {df['M2_Sharpe'].mean():.4f}")
    else:
        print(f"\n{name}: No data (tickers not available)")


RÉSUMÉ FINAL

Table 2&3 (Main):
  Pairs: 2
  Mean M1 Sharpe: 0.2914
  Mean M2 Sharpe: 1.1222

Table A1 (Large Banks):
  Pairs: 10
  Mean M1 Sharpe: 0.4812
  Mean M2 Sharpe: 1.1194

Table A1 (Small Banks):
  Pairs: 10
  Mean M1 Sharpe: 0.4681
  Mean M2 Sharpe: 1.3026

Table A2 (Large×Small):
  Pairs: 25
  Mean M1 Sharpe: 0.6325
  Mean M2 Sharpe: 1.1652

Table A3 IS:
  Pairs: 10
  Mean M1 Sharpe: 0.5763
  Mean M2 Sharpe: 1.0277

Table A3 OOS:
  Pairs: 10
  Mean M1 Sharpe: 1.1330
  Mean M2 Sharpe: 1.6927

Table A4 IS:
  Pairs: 10
  Mean M1 Sharpe: 0.6450
  Mean M2 Sharpe: 1.3501

Table A4 OOS:
  Pairs: 10
  Mean M1 Sharpe: 1.1632
  Mean M2 Sharpe: 1.4233

Table A5 (IS):
  Pairs: 25
  Mean M1 Sharpe: 0.6932
  Mean M2 Sharpe: 1.2616

Table A6 (OOS):
  Pairs: 25
  Mean M1 Sharpe: 1.0181
  Mean M2 Sharpe: 1.7702
